<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/data-extraction-works/01-tabulapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reference**

https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/

https://datatofish.com/export-pandas-dataframe-json/

In [ ]:
!pip install -q tabula-py

In [101]:
import tabula
import pandas as pd

##Loading pdf into Pandas

In [322]:
# Read pdf into DataFrame
dfs = tabula.read_pdf("redacted1.pdf", lattice=True, relative_area=False, multiple_tables=True, pages="all")

# convert PDF into CSV
#tabula.convert_into("redacted1.pdf", "output.csv", output_format="csv")

Oct 19, 2022 11:18:35 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [323]:
len(dfs)

207

In [175]:
dfs[0].columns[0]

'Unnamed: 0'

In [191]:
data_df = []
for idx in range(len(dfs)):
  # filter unwanted tables
  if (dfs[idx].columns[0] != "Unnamed: 0") and (dfs[idx].columns[0] != "Condition") and (dfs[idx].columns[0] != "Response Status"):
    data_df.append(dfs[idx])

In [324]:
data_df = []
for idx in range(len(dfs)):
  # filter unwanted tables
  if dfs[idx].columns[0] in ["Test Name", "Results", "Reference Range", "Score", "Encounter"]:
    data_df.append(dfs[idx])

In [325]:
len(data_df)

86

In [326]:
all_data_df = pd.concat(data_df)

In [327]:
all_data_df.reset_index(drop=True, inplace=True)

In [328]:
all_data_df.columns

Index(['Encounter', 'Specimen', 'Service Date', 'High Score', 'Unnamed: 0',
       'Unnamed: 1', 'Test Name', 'Results', 'Reference Range', 'Score',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Doctor Name', 'Location', 'Phone', 'Lab'],
      dtype='object')

In [329]:
all_data_df.head()

,Encounter,Specimen,Service Date,High Score,Unnamed: 0,Unnamed: 1,Test Name,Results,Reference Range,Score,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Doctor Name,Location,Phone,Lab
0,30,NaN,2021-09-24,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Diagnosis Code,Description,Code Type,Score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D649,"Anemia, unspecified",ICD10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E611,Iron deficiency,ICD10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Save Dataframe into JSON file

In [330]:
# drop unwanted columns
data_df = all_data_df[all_data_df.columns.drop(list(all_data_df.filter(regex='Unnamed')))]
data_df = data_df[data_df.columns.drop(list(data_df.filter(regex='Lab')))]
data_df = data_df[data_df.columns.drop(list(data_df.filter(regex='Location')))]
data_df = data_df[data_df.columns.drop(list(data_df.filter(regex='Phone')))]
data_df = data_df[data_df.columns.drop(list(data_df.filter(regex='Doctor Name')))]
data_df.shape

(1040, 8)

In [331]:
# split dataframe
encounter_df_columns = ["Encounter", "Specimen", "Service Date", "High Score"]
test_df_columns = ["Test Name",	"Results", "Reference Range",	"Score"]
test_data_df = data_df[test_df_columns]
encounter_data_df = data_df[encounter_df_columns]
(test_data_df.shape, encounter_data_df.shape)

((1040, 4), (1040, 4))

In [332]:
# Drop row that has all NaN values
test_data_df= test_data_df.dropna(how="all", axis=0)
encounter_data_df= encounter_data_df.dropna(how="all", axis=0)
(test_data_df.shape, encounter_data_df.shape)

((767, 4), (224, 4))

In [333]:
encounter_data_df.head(20)

,Encounter,Specimen,Service Date,High Score
0,30,NaN,2021-09-24,5
2,Diagnosis Code,Description,Code Type,Score
3,D649,"Anemia, unspecified",ICD10,5
4,E611,Iron deficiency,ICD10,5
5,I951,Orthostatic hypotension,ICD10,5
6,N920,Excessive and frequent menstruation with regul...,ICD10,5
7,R197,"Diarrhea, unspecified",ICD10,5
8,R7889,"Finding of other specified substances, not nor...",ICD10,5
9,Test Name,Results,Reference Range,Score
10,GLUCOSE,100 H,65-99 mg/dL,5


In [ ]:
encounter_data_df['High Score'][]

In [334]:
# Drop all rows with column 'Test Name' and Reference Range has value 'Test Name', 'Diagnosis Code' and 'ICD10'
index_encounter_value = encounter_data_df[(encounter_data_df['Encounter'] == 'Diagnosis Code') | (encounter_data_df['Service Date'] == 'ICD10') | 
                                          (encounter_data_df['Service Date'] == 'ICD9')].index
index_test_value = test_data_df[(test_data_df['Test Name'] == 'Diagnosis Code') | 
                                (test_data_df['Reference Range'] == 'ICD10')].index
encounter_data_df = encounter_data_df.drop(index_encounter_value)
test_data_df = test_data_df.drop(index_test_value)
(test_data_df.shape, encounter_data_df.shape)

((740, 4), (139, 4))

In [335]:
encounter_data_df.head(20)

,Encounter,Specimen,Service Date,High Score
0,30,NaN,2021-09-24,5
9,Test Name,Results,Reference Range,Score
10,GLUCOSE,100 H,65-99 mg/dL,5
11,WHITE BLOOD CELL COUNT,3.2 L,3.8-10.8 Thousand/uL,5
12,RDW,15.6 H,11.0-15.0 %,5
13,ABSOLUTE NEUTROPHILS,1459 L,1500-7800 cells/uL,5
14,26.0,NaN,2021-08-17,5.0
138,10.0,NaN,2019-02-19,10.0
179,2,NaN,2016-10-06,10
181,Doctor Specialty,Score,NaN,NaN


In [338]:
# save to json file
encounter_data_df.to_json('test1.json', orient="columns")

In [339]:
!cat test1.json

{"Encounter":{"0":"30","9":"Test Name","10":"GLUCOSE","11":"WHITE BLOOD CELL COUNT","12":"RDW","13":"ABSOLUTE NEUTROPHILS","14":26.0,"138":10.0,"179":"2","181":"Doctor Specialty","182":"Psychiatry & Neurology - Psychiatry","187":"Test Name","188":"Hemoglobin","189":"Hematocrit","190":"MCH","191":"Encounter","192":"1","194":"Doctor Specialty","195":"Neurology","196":"Neuropsychiatry","197":"Psychiatry","198":"Substance Abuse","202":"Test Name","203":"AST","204":"VITAMIN D,25-OH,TOTAL,IA","205":"30","207":"Doctor Specialty","208":"Internal Medicine","259":"29","261":"Doctor Specialty","262":"Gastroenterology","263":"General Practice","264":"Hospitalist","265":"Internal Medicine","273":"28","275":"Doctor Specialty","276":"Gastroenterology","277":"General Practice","278":"Hospitalist","279":"Internal Medicine","288":"27","290":"Doctor Specialty","291":"Obstetrics & Gynecology","297":26.0,"326":"25","328":"Doctor Specialty","329":"Internal Medicine","362":"24","364":"Doctor Specialty","365"

In [342]:
test_data_df.head(20)

,Test Name,Results,Reference Range,Score
16,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
17,RDW,16.5 H,11.0-15.0 %,5
18,Encounter,Specimen,Service Date,High Score
19,25,NaN,2021-06-23,5
25,Test Name,Results,Reference Range,Score
26,"IRON, TOTAL",31 L,40-190 mcg/dL,5
27,% SATURATION,8 L,16-45 % (calc),5
28,FERRITIN,3 L,16-154 ng/mL,5
29,RED BLOOD CELL COUNT,3.75 L,3.80-5.10 Million/uL,5
30,HEMOGLOBIN,10.3 L,11.7-15.5 g/dL,5


In [340]:
# save to json file
test_data_df.to_json('test2.json', orient="columns")

In [341]:
!cat test2.json

{"Test Name":{"16":"HEMOGLOBIN","17":"RDW","18":"Encounter","19":"25","25":"Test Name","26":"IRON, TOTAL","27":"% SATURATION","28":"FERRITIN","29":"RED BLOOD CELL COUNT","30":"HEMOGLOBIN","31":"HEMATOCRIT","32":"MCHC","33":"Encounter","34":"24","39":"Test Name","40":"RED BLOOD CELL COUNT","41":"HEMOGLOBIN","42":"HEMATOCRIT","43":"ABSOLUTE NEUTROPHILS","44":"Encounter","45":"23","47":"LEUKOCYTE ESTERASE","48":"Encounter","49":"22","54":"Test Name","55":"HEMOGLOBIN","56":"ABSOLUTE NEUTROPHILS","57":"BASOPHILS","58":"Encounter","59":"21","63":"D649","64":"N920","65":"Test N","66":"WHITE BLOOD CELL COUNT","67":"HEMOGLOBIN","68":"HEMATOCRIT","69":"Encounter","70":"18","72":"CASHEW NUT (F202) IGG","73":"WALNUT (F256) IGG","74":"GLUTEN (F79) IGG","75":"BETA LACTOGLOBULIN (F77) IGG","76":"WHEAT (F4) IGG","77":"TOMATO (F25) IGG","78":"EGG WHITE (F1) IGG","79":"PEANUT (F13) IGG","80":"SOYBEAN (F14) IGG","81":"EGG YOLK (F75) IGG","82":"CASEIN (F78) IGG","83":"PEA (F12) IGG","84":"MAIZE\/CORN (F8)

In [70]:
data_df["Results"].astype(str) == "nan"

0     False
1     False
2     False
3      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18     True
24    False
25    False
26    False
27    False
28    False
29     True
Name: Results, dtype: bool

In [72]:
# splitting dataframe by groups
grouped = data_df.groupby(data_df["Results"].astype(str))
service_date_df = grouped.get_group("nan")
service_date_df

,Test Name,Results,Reference Range,Score
3,25,NaN,2021-06-23,5
18,24,NaN,2021-06-03,5
29,23,NaN,2021-04-29,5


In [ ]:
# save to json file
out_file = data_df.to_json('test.json', date_format='iso', orient='records')

In [18]:
# save to json file
out_file = data_df.to_json('test.json', orient="columns")

In [19]:
!cat test.json

{"Test Name":{"0":"HEMOGLOBIN","1":"RDW","2":"Encounter","3":"25","5":"Diagnosis Code","6":"D649","7":"D709","8":"Z79899","9":"Test Name","10":"IRON, TOTAL","11":"% SATURATION","12":"FERRITIN","13":"RED BLOOD CELL COUNT","14":"HEMOGLOBIN","15":"HEMATOCRIT","16":"MCHC","17":"Encounter","18":"24","20":"Diagnosis Code","21":"D649","22":"D709","23":"Test Name","24":"RED BLOOD CELL COUNT","25":"HEMOGLOBIN","26":"HEMATOCRIT","27":"ABSOLUTE NEUTROPHILS","28":"Encounter","29":"23"},"Results":{"0":"11.3 L","1":"16.5 H","2":"Specimen","3":null,"5":"Description","6":"Anemia, unspecified","7":"Neutropenia, unspecified","8":"Other long term (current) drug therapy","9":"Results","10":"31 L","11":"8 L","12":"3 L","13":"3.75 L","14":"10.3 L","15":"32.6 L","16":"31.6 L","17":"Specimen","18":null,"20":"Description","21":"Anemia, unspecified","22":"Neutropenia, unspecified","23":"Results","24":"3.63 L","25":"10.1 L","26":"31.3 L","27":"1428 L","28":"Specimen","29":null},"Reference Range":{"0":"11.7-15.5 

In [ ]:
def save_data_into_json(df, idx):
  # drop column that has named "Unnamed: xxx"
  data_df = df[df.columns.drop(list(df.filter(regex="Unnamed")))]
  # Drop row that has all NaN values
  data_df= data_df.dropna(how="all", axis=0)
  # save to json file
  data_df.to_json(f"data/test-{idx}.json", date_format="iso", orient="records")

In [ ]:
!mkdir data

In [ ]:
for idx in range(len(dfs)):
  if dfs[idx].columns[0] != "Unnamed: 0":
    save_data_into_json(dfs[idx], idx)

##Further practice

In [ ]:
tabula.read_pdf("redacted1.pdf", pages="all", stream=True)

In [ ]:
dfs = tabula.read_pdf(
    "redacted1.pdf",
    lattice=True,
    #pandas_options={"header": [0, 1]},
    # area=[0, 0, 50, 100],
    relative_area=True,
    multiple_tables=True,
    pages=1,
)

Oct 18, 2022 10:37:59 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
df = dfs[0].drop(["Unnamed: 0"], axis=1)
df

""


In [ ]:
len(df)

0

In [ ]:
dfs[9]

,Unnamed: 0,Unnamed: 1
0,NaN,NaN


In [ ]:
len(dfs[6].value_counts())

4

In [ ]:
data_idx = []
for idx in range(len(dfs)):
  if len(dfs[idx].value_counts()) > 0:
    data_idx.append(idx)

In [ ]:
data_idx

[6, 7]

In [ ]:
dfs[8]

,Encounter,Specimen,Service Date,High Score,Unnamed: 0,Unnamed: 1
0,30,NaN,2021-09-24,5,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Diagnosis Code,Description,Code Type,Score,NaN,NaN
3,D649,"Anemia, unspecified",ICD10,5,NaN,NaN
4,E611,Iron deficiency,ICD10,5,NaN,NaN
5,I951,Orthostatic hypotension,ICD10,5,NaN,NaN
6,N920,Excessive and frequent menstruation with regul...,ICD10,5,NaN,NaN
7,R197,"Diarrhea, unspecified",ICD10,5,NaN,NaN
8,R7889,"Finding of other specified substances, not nor...",ICD10,5,NaN,NaN
9,Test Name,Results,Reference Range,Score,NaN,NaN


In [ ]:
data_df = None
for idx in range(len(dfs)):
  if len(dfs[idx].value_counts()) > 0:
    data_df = dfs[idx]
data_df

,Condition,Provider\rGenerated,Demographic\rPrevalence,Confidence,Contributing Factors
0,Oth symptoms and signs w\rcognitive functions ...,False,1.4%,99.0%,"Amorphous Sediment [Presence],Platelet\r[Prese..."
1,Any Muscle Disease,True,43.5%,89.5%,"R29-Nervous Systom\rSymp.,Sodium,Hydromorphone..."
2,Abnormalities of breathing,False,5.7%,57.2%,"LE,HDL,Free T4 Index,GFR - MDRD"
3,Other anemias,True,4.0%,52.2%,"Total Bilirubin,ALP,Hemoglobin,D50-Anemia(Iron)"


In [ ]:
tabula.convert_into("redacted1.pdf", "test.json", output_format="json")

Oct 18, 2022 9:53:01 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
!cat test.json

[{"extraction_method":"stream","top":249.0,"left":41.0,"width":523.0,"height":116.0,"right":564.0,"bottom":365.0,"data":[[{"top":256.81,"left":71.71,"width":48.05000305175781,"height":4.630000114440918,"text":"Condition"},{"top":256.81,"left":161.93,"width":43.350006103515625,"height":4.630000114440918,"text":"Provider"},{"top":256.81,"left":222.26,"width":66.68000793457031,"height":4.630000114440918,"text":"Demographic"},{"top":256.81,"left":300.38,"width":55.839996337890625,"height":4.630000114440918,"text":"Confidence"},{"top":256.81,"left":414.65,"width":98.8900146484375,"height":4.630000114440918,"text":"Contributing Factors"}],[{"top":0.0,"left":0.0,"width":0.0,"height":0.0,"text":""},{"top":267.97,"left":158.9,"width":50.850006103515625,"height":4.630000114440918,"text":"Generated"},{"top":267.97,"left":229.18,"width":52.810001373291016,"height":4.630000114440918,"text":"Prevalence"},{"top":0.0,"left":0.0,"width":0.0,"height":0.0,"text":""},{"top":0.0,"left":0.0,"width":0.0,"hei

In [ ]:
tabula.convert_into("redacted1.pdf", "test.tsv", output_format="tsv", 
                    pages='all', lattice=True, area=[0, 0, 50, 100],
                    relative_area=True,
                    )

Oct 18, 2022 9:58:42 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
!cat test.tsv

""
""
""
""
""
""
Prevalence"	Confidence	Contributing Factors
[Presence],Methadone,ESR"
Symp.,Sodium,Hydromorphone,Free T4"
Abnormalities of breathing	False	5.7%	57.2%	LE,HDL,Free T4 Index,GFR - MDRD
Other anemias	True	4.0%	52.2%	Total Bilirubin,ALP,Hemoglobin,D50-Anemia(Iron)
""
Encounter	Specimen	Service Date	High Score		
26		2021-08-17	5		
""					
""	
""	
Test Name	Results	Reference Range	Score		
HEMOGLOBIN	11.3 L	11.7-15.5 g/dL	5		
RDW	16.5 H	11.0-15.0 %	5		
Encounter	Specimen	Service Date	High Score		
25		2021-06-23	5		
""					
Diagnosis Code	Description	Code Type	Score		
D649	Anemia, unspecified	ICD10	5		
D709	Neutropenia, unspecified	ICD10	5		
Z79899	Other long term (current) drug therapy	ICD10	5		
Test Name	Results	Reference Range	Score		
IRON, TOTAL	31 L	40-190 mcg/dL	5		
% SATURATION	8 L	16-45 % (calc)	5		
""	
""	
""
Test Name	Results	Reference Range	Score		
LEUKOCYTE ESTERASE	TRACE A	NEGATIVE	5		
Encounter	Specimen	Service Date	High Score		
22		2021-03-31	10		
""					
Diagno

In [ ]:
import pandas as pd
import json

In [ ]:
master_df = pd.read_csv('test.tsv', sep='\s+', encoding='utf-8', on_bad_lines='skip')

In [ ]:
out_file = master_df.to_json('test.json', date_format='iso', orient='records')

In [ ]:
!cat test.json

[{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":"21"},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null},{"Unnamed: 0":null}